We find the disallowed region in parameter space where the too-low-SFR constraint is triggered. i.e. *The total SFR of the last age bin is below 5% of the mean SFR*

In [95]:
%pylab inline
import os
os.chdir('/home/oliverphilcox/ChempyMulti/')
from Chempy.sfr import SFR
from Chempy.parameter import ModelParameters
a=ModelParameters()
age_of_star=0.

Populating the interactive namespace from numpy and matplotlib


/home/oliverphilcox/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['uniform', 'diff', 'multivariate_normal']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [15]:
def fraction_sfr(log_sfr_param):
    sfr_scale=10.**log_sfr_param
    basic_sfr = SFR(a.start,a.end,a.time_steps)
    if a.basic_sfr_name == 'gamma_function':
        getattr(basic_sfr, a.basic_sfr_name)(S0 = a.S_0 * a.mass_factor,a_parameter = a.a_parameter, loc = a.sfr_beginning, scale = sfr_scale)
    basic_sfr.sfr = a.total_mass * np.divide(basic_sfr.sfr,sum(basic_sfr.sfr))
    mass_normalisation = a.total_mass
    mean_sfr = sum(basic_sfr.sfr) / a.end


    # at which time in the simulation is the star born
    star_time = basic_sfr.t[-1] - age_of_star
    cut = [np.where(np.abs(basic_sfr.t - star_time) == np.min(np.abs(basic_sfr.t - star_time)))]
    if len(cut[0][0]) != 1:
        cut = cut[0][0][0]

    # updating the end time and the model steps and rescale the total mass
    time_model = float(basic_sfr.t[cut])
    a.end = time_model
    a.time_steps = int(cut[0][0]) + 1
    a.total_mass = sum(basic_sfr.sfr[0:a.time_steps])

    # check whether the sfr is enough at end to produce reasonable number of stars (which is necessary in order to have a probability to observe a star at all)
    sfr_at_end = float(basic_sfr.sfr[cut] / basic_sfr.dt)
    fraction_of_mean_sfr = sfr_at_end / mean_sfr

    return(fraction_of_mean_sfr)

In [36]:
mmin=0.1;mmax=1.
for _ in range(20):
    diff=(mmax-mmin)/10
    params=np.arange(mmin,mmax,diff)
    outs=np.asarray([fraction_sfr(p) for p in params])
    new_start=min(params[np.where(outs>0.05)[0]])
    mmax=new_start+diff
    mmin=new_start-diff
    print(new_start)

0.37
0.29800000000000004
0.2944
0.2944
0.29411199999999993
0.29402559999999994
0.29401984000000003
0.294016384
0.29401546240000004
0.29401537023999996
0.29401535180800004
0.2940153444352
0.29401534406656005
0.29401534399283213
0.2940153439928321
0.29401534398398477
0.29401534398162543
0.29401534398127155
0.2940153439811772
0.294015343981163


/home/oliverphilcox/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/home/oliverphilcox/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


Thus we need $\log_{10}\text{SFR}_\mathrm{peak}>0.294$ for sensible star-formation histories. This will be reflected in the priors.